# Задача: найти оптимальные настройки для минимизации использования полосы пропускания при сохранении приемлемого времени конвергенции

In [2]:
import random
import time

class Node:
    def __init__(self, node_id):
        self.id = node_id
        self.knows_failure = False

class BaseSimulator:
    def __init__(self, num_nodes, interval, node_failures):
        self.nodes = [Node(i) for i in range(num_nodes)]
        self.interval = interval
        self.node_failures = node_failures
        self.failed_nodes = set()
        self.bandwidth_usage = 0

    def simulate_failure(self):
        num_failures = int(len(self.nodes) * self.node_failures / 100)
        self.failed_nodes = set(random.sample(range(len(self.nodes)), num_failures))
        if self.failed_nodes:
            self.nodes[random.choice(list(self.failed_nodes))].knows_failure = True
        elif self.nodes:
            self.nodes[0].knows_failure = True

    def run_simulation(self, max_time=5):
        self.simulate_failure()
        start_time = time.time()
        first_knowledge_time = None
        all_knowledge_time = None

        while True:
            self.detect_failures()
            current_time = time.time() - start_time

            if first_knowledge_time is None and any(node.knows_failure for node in self.nodes if node.id not in self.failed_nodes):
                first_knowledge_time = current_time

            if all(node.knows_failure for node in self.nodes if node.id not in self.failed_nodes):
                all_knowledge_time = current_time
                break

            if current_time > max_time:  # Ограничение времени симуляции
                break

            time.sleep(self.interval)

        return first_knowledge_time or max_time, all_knowledge_time or max_time, self.bandwidth_usage

class SerfSimulator(BaseSimulator):
    def __init__(self, num_nodes, gossip_interval, gossip_fanout, packet_loss, node_failures):
        super().__init__(num_nodes, gossip_interval, node_failures)
        self.gossip_fanout = gossip_fanout
        self.packet_loss = packet_loss

    def detect_failures(self):
        for node in self.nodes:
            if node.id not in self.failed_nodes and node.knows_failure:
                active_nodes = [n for n in range(len(self.nodes)) if n != node.id and n not in self.failed_nodes]
                if not active_nodes:
                    continue
                targets = random.sample(active_nodes, min(self.gossip_fanout, len(active_nodes)))
                for target in targets:
                    if random.random() > self.packet_loss / 100:
                        self.nodes[target].knows_failure = True
                        self.bandwidth_usage += 1

def find_optimal_settings(num_nodes, time_threshold, fast_mode=True):
    # Узкие диапазоны для быстрого выполнения
    intervals = [0.5, 1.0] if fast_mode else [0.2, 0.5, 1.0]
    fanouts = [2, 3] if fast_mode else [2, 3, 5]
    packet_losses = [0, 5] if fast_mode else [0, 5, 10, 25]
    node_failures_set = [5] if fast_mode else [0, 5, 10]

    optimal_settings = []

    for gossip_interval in intervals:
        for gossip_fanout in fanouts:
            for packet_loss in packet_losses:
                for node_failures in node_failures_set:
                    simulator = SerfSimulator(num_nodes, gossip_interval, gossip_fanout, packet_loss, node_failures)
                    result = simulator.run_simulation(max_time=time_threshold)

                    first_knowledge_time, all_knowledge_time, bandwidth_usage = result

                    if all_knowledge_time <= time_threshold:  # Проверка на соответствие времени
                        optimal_settings.append((gossip_interval, gossip_fanout, packet_loss, node_failures, bandwidth_usage))

    # Сортировка по использованию полосы пропускания
    optimal_settings.sort(key=lambda x: x[4])

    return optimal_settings

# Запуск поиска оптимальных настроек
num_nodes = 50  # Число узлов
time_threshold = 5  # Максимальное допустимое время конвергенции
fast_mode = True  # Быстрый режим, если нужен ускоренный расчет

optimal_results = find_optimal_settings(num_nodes, time_threshold, fast_mode)

# Вывод результатов
print("Оптимальные настройки для минимизации использования полосы пропускания:")
for settings in optimal_results:
    print(f"Gossip Interval: {settings[0]}, Gossip Fanout: {settings[1]}, Packet Loss: {settings[2]}%, Node Failures: {settings[3]}%, Bandwidth Usage: {settings[4]}")


Оптимальные настройки для минимизации использования полосы пропускания:
Gossip Interval: 0.5, Gossip Fanout: 2, Packet Loss: 0%, Node Failures: 5%, Bandwidth Usage: 0
Gossip Interval: 0.5, Gossip Fanout: 2, Packet Loss: 5%, Node Failures: 5%, Bandwidth Usage: 0
Gossip Interval: 0.5, Gossip Fanout: 3, Packet Loss: 0%, Node Failures: 5%, Bandwidth Usage: 0
Gossip Interval: 0.5, Gossip Fanout: 3, Packet Loss: 5%, Node Failures: 5%, Bandwidth Usage: 0
Gossip Interval: 1.0, Gossip Fanout: 2, Packet Loss: 0%, Node Failures: 5%, Bandwidth Usage: 0
Gossip Interval: 1.0, Gossip Fanout: 2, Packet Loss: 5%, Node Failures: 5%, Bandwidth Usage: 0
Gossip Interval: 1.0, Gossip Fanout: 3, Packet Loss: 0%, Node Failures: 5%, Bandwidth Usage: 0
Gossip Interval: 1.0, Gossip Fanout: 3, Packet Loss: 5%, Node Failures: 5%, Bandwidth Usage: 0
